In [14]:
import numpy as np
import sys
from sys import argv
import gdal, gdalconst
import math
import os
from osgeo import osr
# TODO: add directory crawler to be able to move through downloaded image sets (once they're downloaded)
# TODO: add identifier to array from folder name to associated files (whether it's landsat5,6,7,8)
# factor out different calculations based on satellite
# add nuvi calculations as other methods
# auto-clip input files (either to file or in memory) to aoi shapefile

# Type for internal calculations:
t = np.float64

dir = 'C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/'
rr = "C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_band4.tif"
red = gdal.Open("C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_band4.tif")
nir = gdal.Open("C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_band5.tif")
blue = gdal.Open("C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_band2.tif")
meta = 'C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_MTL.txt' # meta text file for corresponding raster to pull correction factors from

geotiff = gdal.GetDriverByName('GTiff')
outfile_NDVI = "C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_NDVIaaaaaaaaa.tif"
outfile_EVI = "C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_EVI.tif"
outfile_SAVI = "C:/Users/marioag/Documents/GitHub/telescope/python_notebooks/LC08_L1TP_007069_20180411_20180417_01_T1_sr_SAVI.tif"


# Ugly syntax, but fast:
r = red.GetRasterBand(1).ReadAsArray(0, 0, red.RasterXSize, red.RasterYSize)
n = nir.GetRasterBand(1).ReadAsArray(0, 0, nir.RasterXSize, nir.RasterYSize)
b = blue.GetRasterBand(1).ReadAsArray(0, 0, blue.RasterXSize, blue.RasterYSize)
# print type(b)
# Convert the 16-bit Landsat 8 values to floats for the division operation:
r = r.astype(t)
n = n.astype(t)
b = b.astype(t)

# Tell numpy not to complain about division by 0:
np.seterr(invalid='ignore')


tr = []
tr =  red.GetGeoTransform()
# print tr
# print type(tr)

def getRasterInfo(raster):
    source = gdal.Open(raster)
    transformation = source.GetGeoTransform()
    pixelWidth = transformation[1]
    pixelHeight = transformation[5]
    originX = transformation[0]
    originY = transformation[3]
    projection = osr.SpatialReference()
    projection.ImportFromWkt(source.GetProjectionRef())
    xSize = source.RasterXSize
    ySize = source.RasterYSize
    # print projection
    return transformation, projection, xSize, ySize, pixelWidth, pixelHeight, originX, originY

def getSunElevation(meta):
    f = open(os.path.abspath(meta), 'r')
    for line in f.readlines():
        if '    SUN_ELEVATION =' in line:
            sunElev = line.split('=')[1]
    return float(sunElev)
    f.close()

def applyCorrection(x, meta):
    xCor = ((math.e**-5)*x) + -0.100000 
    print xCor
    xCor = xCor/math.sin(getSunElevation(meta))
#     print xCor
    return xCor
    # open file
    # apply transformation as defined in file
    # return arrays to be used in ndvi function
    
def ndvi(r, n, meta, outfile, red):
    nC = applyCorrection(n, meta)
    rC = applyCorrection(r, meta)
    ndvi = (nC-rC)/(nC+rC)
    transformation, projection, xSize, ySize, pixelWidth, pixelHeight, originX, originY = getRasterInfo(red)
    output = geotiff.Create(outfile, xSize, ySize, 1, gdal.GDT_Float64)
    print type(output)
    output.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    output.SetProjection(projection.ExportToWkt())
    output.GetRasterBand(1).WriteArray(ndvi)
    output.FlushCache()
    print "Calculated NDVI"
    return output

def evi(r,n,b, meta, outfile, red):
    nC = n #applyCorrection(n, meta)
    rC = r #applyCorrection(r, meta)
    bC = b #applyCorrection(b, meta)
    evi = 2.5* (((nC - rC)) / (nC + 6 * rC - 7.5 * bC - 1))
    transformation, projection, xSize, ySize, pixelWidth, pixelHeight, originX, originY = getRasterInfo(red)
    output = geotiff.Create(outfile, xSize, ySize, 1, gdal.GDT_Float64)
    print type(output)
    output.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    output.SetProjection(projection.ExportToWkt())
    output.GetRasterBand(1).WriteArray(evi)
    output.FlushCache()
    return output

def savi(r,n, b, outfile, red):
    nC = n #applyCorrection(n, meta)
    rC = r #applyCorrection(r, meta)
    bC = b #applyCorrection(b, meta)
    savi = (nC - rC) / (nC + rC + 0.5) * 0.5
    transformation, projection, xSize, ySize, pixelWidth, pixelHeight, originX, originY = getRasterInfo(red)
    output = geotiff.Create(outfile, xSize, ySize, 1, gdal.GDT_Float64)
    output.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    output.SetProjection(projection.ExportToWkt())
    output.GetRasterBand(1).WriteArray(savi)
    output.FlushCache()
    return output

def dirNDVI(dir):
    files = []
    for each in os.listdir(dir):
        if each.startswith('LC08'):
            files.append(each[:41])
    base_files = []
    for every in files:
        if every not in base_files:
            base_files.append(every)
    os.chdir(dir)
    for image in base_files:
        ff = image + 'sr_band4.tif'
#         ff = str(os.path.abspath(ff).replace("\\","/"))
        print ff
        red = gdal.Open(ff, 1)
#         print type(Red)
        nir = image + 'sr_band5.tif'
        nir = gdal.Open(nir, 1)
#         nir = gdal.Open(os.path.abspath(nir))
        print nir
        meta = os.path.abspath(image+'MTL.txt')
        print meta
        outfile = os.path.abspath(image+'_ndviout.tif')
        r = red.GetRasterBand(1).ReadAsArray(0, 0, red.RasterXSize, red.RasterYSize).astype(t)
        n = nir.GetRasterBand(1).ReadAsArray(0, 0, nir.RasterXSize, nir.RasterYSize).astype(t)
        ndvi(r, n, meta, outfile)
    return base_files

# dirNDVI(dir)
# print getSunElevation(meta)
# applyCorrection(r)

ndvi = ndvi(r, n, meta, outfile_NDVI, rr)
# ndvi.SetGeoTransform(tr)

evi = evi(r,n,b, meta, outfile_EVI, rr)
# evi.SetGeoTransform(tr)


savi = savi(r,n, meta, outfile_SAVI, rr)
# savi.SetGeoTransform(tr)

print "finished"

# The ndvi value is in the range -1..1, but we want it to be displayable, so:
# Make the value positive and scale it back up to the 16-bit range:
# ndvi = (ndvi + 1) * (2**15 - 1)

# And do the type conversion back:
# ndvi = ndvi.astype(np.uint16)
# output.SetProjection(red.GetProjection())

# output.GetRasterBand(1).WriteArray(ndvi)
# exit()

[[-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 ...
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]]
[[-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 ...
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 [-67.47273204 -67.47273204 -67.47273204 ... -67.47273204 -67.47273204
  -67.47273204]
 [-67.47273204 -67.47273204 -67.

c:\users\marioag\anaconda2\envs\telescope\lib\site-packages\ipykernel_launcher.py:99: RuntimeWarning: divide by zero encountered in divide


<class 'osgeo.gdal.Dataset'>
finished


In [8]:
### Arcpy snippet to clip all layers by the cleaned bounding geometry- just parking it here for reference, it was run from the arc console

path = ""

for file in os.listdir(path):
    if file.endswith(".tif"):
        out = path + '/clipped/' + file + "_clipped.tif"
        arcpy.Clip_management(file, rectangle, out, in_template_dataset="greenbelt_buffers_union", clipping_geometry="ClippingGeometry")
        



WindowsError: [Error 3] The system cannot find the path specified: ''